In [1]:
import os, sys, time, resource, re, gc, shutil
from multiprocess import Pool
from functools import partial
from tabulate import tabulate
from urllib.parse import urlparse, parse_qsl

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mongoengine.queryset.visitor import Q
import django
from django.db.models import Count, Sum
sys.path.append('/home/galm/software/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from django.forms.models import model_to_dict

from scoping.models import *
from tmv_app.models import *
from scipy import stats
import ipy_table as tbl

run_id = 181
qid = 1281

In [2]:
topics = Topic.objects.filter(run_id=181)
dts = DocTopic.objects.filter(run_id=181)

p1total = dts.filter(doc__PY__gt=2009,doc__PY__lt=2014).aggregate(t=Sum('score'))
p2total = dts.filter(doc__PY__gt=2013,doc__PY__lt=2018).aggregate(t=Sum('score'))

In [3]:
p1total

{'t': 304.886883627571}

In [4]:
p2total

{'t': 583.587515019855}

In [5]:
topic_objects = []
for t in topics:
    t1 = dts.filter(doc__PY__gt=2007,doc__PY__lt=2013,topic=t).aggregate(t=Sum('score'))
    t2 = dts.filter(doc__PY__gt=2012,doc__PY__lt=2018,topic=t).aggregate(t=Sum('score'))
    p1 = t1['t'] / p1total['t']
    p2 = t2['t'] / p2total['t']
    growth = (p2 - p1) / p1*100
    t.growth = float(growth)
    t.save()
    print(t.title)
    print(t.growth)
    t.save()
    topic_objects.append(t)
    
print(topic_objects)
for top in topic_objects:
    print(top.growth)
    print(top.run_id_id)
    top.save()
    
#Topic.objects.update(topics)
t.save()

{polici, instrument, effect}
-4.9418958237658215
{urban, land, citi}
18.371650553346157
{park, space, price}
26.766112317544593
{transit, rail, station}
4.8065313829059795
{vehicl, electr, charg}
41.693567913840354
{recycl, household, collect}
-8.33511366097385
{transport, public, access}
-11.893397322892232
{water, heat, suppli}
7.708097861229543
{bus, servic, oper}
-1.7999086155463713
{green, build, roof}
89.22278266721857
{climat, chang, mitig}
-4.977116920193816
{heat, thermal, comfort}
6.31183481875969
{toll, lane, link}
-35.17002524376201
{brt, bus, system}
33.71503801982712
{activ, school, health}
-8.464208504308415
{air, pollut, qualiti}
-12.932083643569362
{model, network, optim}
6.311566372862173
{treatment, wastewat, plant}
-21.164350035350164
{emiss, carbon, ghg}
4.352767248848828
{bicycl, cycl, cyclist}
11.709834064989696
{travel, mode, choic}
3.763809435567035
{price, congest, road}
-32.9550002251275
{wast, manag, solid}
-35.42498549449721
{energi, build, consumpt}
10.312

In [7]:
topic_table = topics.values('title','growth')
df = pd.DataFrame.from_dict(list(topic_table))
df.to_csv('../data/topic_growth.csv')